# Computing Weather for Each Flight Route

This notebook should have everything you need to compute the weather conditions selected for a given flight.

To make this work, make sure you have the following done:
 - Rename the `config/config_template.yml` file as `config/config.yml`
 - Change the `base-configs - root-directory` entry to wherever you downloaded the project repo
 - Change the `base-configs - tag` entry to whatever you'd like to, I don't think it'll matter, but who knows
 - Make sure all the dependencies are installed with `pip install -r requirements.txt`
 - Change the `flights_database` and `weather_database` variables below to whatever flight one you assigned yourself to, and the corresponding weather
 - Have fun and pray for no bugs!
 - If you need to make an environment readable on Jupyter you can follow [this guide](https://medium.com/@nrk25693/how-to-add-your-conda-environment-to-your-jupyter-notebook-in-just-4-steps-abeab8b8d084). If not, just copy the code and make it into a script. Up to ya

In [15]:
import sys
# This variable should indicate the path from this Jupyter Notebook to the root directory of the repo.
root_path = '../'
# Adds the repo's root to the list of paths
sys.path.append(root_path)

# Package to read yml files
import yaml
# Package to handle file paths
import os
# Package to deal with DataFrames
import pandas as pd
# Package to plot stuff
import matplotlib.pyplot as plt
# Package for numerical and array handling
import numpy as np
# Package to read and write to .sqlite files
import sqlite3
# Package to keep track of time
import datetime

# Function to clear output from jupyter notebook
from IPython.display import clear_output
# Package for compressing dataframes into file
from src.data import compressors
# Package for defining and fitting weather models
from src.models import weather
# Utilities package
from src.common import utils
# Package for interpolating and estimating weather
from src.analysis import weather_interpolator

# Path from this notebook to the root directory
root_path = os.path.normpath(root_path)
# Path from root to the desired config file
config_path_from_root = os.path.normpath('config/config.yml')
# Defining path from this notebook to config file
config_path = os.path.join(root_path, config_path_from_root)

# Loading config file
with open(config_path, 'r',  encoding='utf8') as file:
    config = yaml.safe_load(file)

# Defining "clear-output" function to feed into logger
def clear():
    clear_output(wait=True)

# Creates an instance of a logger class to log all that happens, optional (but encouraged).
logger = utils.Logger(config, clear_function=None)

# Creates an instance of the weather interpolator
interpolator = weather_interpolator.WeatherInterpolator(config, logger=logger)

# Defining location of data
flights_database = '../data/flight/KDEN_KSEA_2023-07-01_2023-07-31.sqlite'
weather_database = '../data/weather/Weather-US_2023-06-30_2022-08-01.sqlite'

# Path to file keeping track of already-loaded flights
tracking_file = flights_database.replace('sqlite','txt')

In [16]:
# Declares the connections read from each of the databases
flights_connection = sqlite3.connect(flights_database)
weather_connection = sqlite3.connect(weather_database)
# Declares a cursor to write to the database
cursor = flights_connection.cursor()

# Checking and loading the tracking_file
if os.path.isfile(tracking_file):
    with open(tracking_file, 'r') as f:
        loaded_ids = f.read().split('\n')
        loaded_ids = [i for i in loaded_ids if i != '']
else:
    with open(tracking_file, 'w') as f:
        loaded_ids = []
    

# Runs a query to identify all the flight_ids available
flight_ids = pd.read_sql_query("SELECT flight_id FROM flights;", flights_connection).values[:,0]
# Only care about the ids that have not been loaded yet.
flight_ids = [f for f in flight_ids if f not in loaded_ids]

# Loading the time threshold variable, which is the time interval that weather data will be loaded for each calculation.
time_thresh = config['statistics']['interpolation']['weather']['time-thresh']
lat_lon_thresh = config['statistics']['interpolation']['weather']['lat-lon-thresh']

# The list of columns to be added to the new table to be created.
new_columns = ['tmpf', 'air_pressure', 'air_density', 'clouds', 'sknt', 'severity']

# If there is no record of loaded ids, we start from scratch
if len(loaded_ids) == 0:
    # Drop the table if it exists
    cursor.execute("DROP TABLE IF EXISTS state_vector_weather;")

    # Create the new table
    create_table_query = f'''
        CREATE TABLE state_vector_weather (
            vector_id INTEGER PRIMARY KEY,
            {", ".join([f"{col} REAL" for col in new_columns])}
        );
    '''
    # Create the new table if it doesn't exist
    cursor.execute(create_table_query)

# Commits change to file
flights_connection.commit()

t_start_full = datetime.datetime.now()
# Looping through flight ids
for i, flight_id in enumerate(flight_ids):
    time_iteration = datetime.datetime.now()
    # Clears output at every loop
    clear_output(wait=True)
    print(f'{flight_id} | {i}/{len(flight_ids)}.')
    time_elapsed = (time_iteration - t_start_full).total_seconds()
    if i == 0:
        ETA = np.nan
    else:
        ETA = time_elapsed*len(flight_ids)/i - time_elapsed
    print(f'Time Elapsed: {utils.format_time(time_elapsed)}.')
    print(f'Estimate time to finish: {utils.format_time(ETA)}.')
    
    # Finds minimum and maximum flight for current flight
    print('Loading time limits.')
    min_time, max_time = pd.read_sql_query(f"""
                                SELECT MIN(time) as min_time, MAX(time) as max_time
                                FROM state_vectors 
                                JOIN flights ON flights.flight_id = state_vectors.flight_id
                                WHERE state_vectors.flight_id = "{flight_id}";
                               """,
                              flights_connection
                              ).values[0]
    
    # Finds the minimum and maximum latitudes for the current flight
    print('Loading latitude limits.')
    min_latitude, max_latitude = pd.read_sql_query(f"""
                                    SELECT MIN(lat) as min_lat, MAX(lat) as max_lat
                                    FROM state_vectors 
                                    JOIN flights ON flights.flight_id = state_vectors.flight_id
                                    WHERE state_vectors.flight_id = "{flight_id}"
                                   """,
                                  flights_connection
                                  ).values[0]

    print('Loading longitude limits.')
    # Finds the minimum and maximum longitudes for the current flight
    min_longitude, max_longitude = pd.read_sql_query(f"""
                                    SELECT MIN(lon) as min_lon, MAX(lon) as max_lon
                                    FROM state_vectors 
                                    JOIN flights ON flights.flight_id = state_vectors.flight_id
                                    WHERE state_vectors.flight_id = "{flight_id}"
                                   """,
                                  flights_connection
                                  ).values[0]
    
    # Adjusting time, lat and lon thresholds.
    print("Adjusting time, lat, lon thresholds.")
    # Adds time threshold to time limits
    min_time -= time_thresh
    max_time += time_thresh
    
    # Adds latitude and longitude threshold
    range_latitude = max_latitude - min_latitude
    range_longitude = max_longitude - min_longitude
    min_latitude -= range_latitude*lat_lon_thresh
    max_latitude += range_latitude*lat_lon_thresh
    min_longitude -= range_longitude*lat_lon_thresh
    max_longitude += range_longitude*lat_lon_thresh    
    
    # Loads the weather data corresponding to the flight
    print('Loading relevant weather data.')
    flight_weather_data = pd.read_sql_query(f"""
                                    SELECT ws.lat, ws.lon, ws.elevation, ws.sigma, wd.*
                                    FROM weather_data as wd
                                    JOIN weather_stations as ws ON ws.station_id = wd.station_id
                                    WHERE wd.time BETWEEN {min_time} AND {max_time};
                                   """,
                                   weather_connection
                                    )

    # Loads state vectors for the given flight
    print('Loading state vectors.')
    state_vectors = pd.read_sql_query(f"""
                                    SELECT DISTINCT state_vectors.*
                                    FROM state_vectors 
                                    JOIN flights ON flights.flight_id = state_vectors.flight_id
                                    WHERE state_vectors.flight_id = "{flight_id}";
                                   """,
                                   flights_connection)
    
    # Computes the weather values for the current flight
    print('Computing weather values.')
    state_vectors = interpolator.compute_flight_weather_quantities(new_columns, state_vectors, stations_data=flight_weather_data)
    
    # Adds newly calculated values to new table    
    print("Adding newly calculated values to new table")
    for index, row in state_vectors.iterrows():
        # Preparing the data to be inserted
        insert_data = tuple(row[col] for col in ['vector_id'] + new_columns)

        # Creating query to insert new values
        insert_query = f'''
            INSERT INTO state_vector_weather (vector_id, {', '.join(new_columns)})
            VALUES ({', '.join('?' * len(insert_data))})
            ON CONFLICT(vector_id) DO UPDATE SET
            {', '.join([f"{col} = excluded.{col}" for col in new_columns])};
        '''
        cursor.execute(insert_query, insert_data)
    # Commiting changes to the database
    flights_connection.commit()
    
    with open(tracking_file, 'a') as f:
        f.write(flight_id + '\n')

# Closing connections
flights_connection.close()
weather_connection.close()

print("Done!")

ac33a3_1690489043_1690497432_KDEN_KSEA | 822/823.
Time Elapsed: 16h 43m 27s.
Estimate time to finish: 1m 13s.
Loading time limits.
Loading latitude limits.
Loading longitude limits.
Adjusting time, lat, lon thresholds.
Loading relevant weather data.
Loading state vectors.
Computing weather values.
Adding newly calculated values to new table
Done!


In [17]:
import sys
# This variable should indicate the path from this Jupyter Notebook to the root directory of the repo.
root_path = '../'
# Adds the repo's root to the list of paths
sys.path.append(root_path)

# Package to read yml files
import yaml
# Package to handle file paths
import os
# Package to deal with DataFrames
import pandas as pd
# Package to plot stuff
import matplotlib.pyplot as plt
# Package for numerical and array handling
import numpy as np
# Package to read and write to .sqlite files
import sqlite3
# Package to keep track of time
import datetime

# Function to clear output from jupyter notebook
from IPython.display import clear_output
# Package for compressing dataframes into file
from src.data import compressors
# Package for defining and fitting weather models
from src.models import weather
# Utilities package
from src.common import utils
# Package for interpolating and estimating weather
from src.analysis import weather_interpolator

# Path from this notebook to the root directory
root_path = os.path.normpath(root_path)
# Path from root to the desired config file
config_path_from_root = os.path.normpath('config/config.yml')
# Defining path from this notebook to config file
config_path = os.path.join(root_path, config_path_from_root)

# Loading config file
with open(config_path, 'r',  encoding='utf8') as file:
    config = yaml.safe_load(file)

# Defining "clear-output" function to feed into logger
def clear():
    clear_output(wait=True)

# Creates an instance of a logger class to log all that happens, optional (but encouraged).
logger = utils.Logger(config, clear_function=None)

# Creates an instance of the weather interpolator
interpolator = weather_interpolator.WeatherInterpolator(config, logger=logger)

# Defining location of data
flights_database = '../data/flight/KJFK_KLAX_2023-07-01_2023-07-31.sqlite'
weather_database = '../data/weather/Weather-US_2023-06-30_2022-08-01.sqlite'

# Path to file keeping track of already-loaded flights
tracking_file = flights_database.replace('sqlite','txt')

In [18]:
# Declares the connections read from each of the databases
flights_connection = sqlite3.connect(flights_database)
weather_connection = sqlite3.connect(weather_database)
# Declares a cursor to write to the database
cursor = flights_connection.cursor()

# Checking and loading the tracking_file
if os.path.isfile(tracking_file):
    with open(tracking_file, 'r') as f:
        loaded_ids = f.read().split('\n')
        loaded_ids = [i for i in loaded_ids if i != '']
else:
    with open(tracking_file, 'w') as f:
        loaded_ids = []
    

# Runs a query to identify all the flight_ids available
flight_ids = pd.read_sql_query("SELECT flight_id FROM flights;", flights_connection).values[:,0]
# Only care about the ids that have not been loaded yet.
flight_ids = [f for f in flight_ids if f not in loaded_ids]

# Loading the time threshold variable, which is the time interval that weather data will be loaded for each calculation.
time_thresh = config['statistics']['interpolation']['weather']['time-thresh']
lat_lon_thresh = config['statistics']['interpolation']['weather']['lat-lon-thresh']

# The list of columns to be added to the new table to be created.
new_columns = ['tmpf', 'air_pressure', 'air_density', 'clouds', 'sknt', 'severity']

# If there is no record of loaded ids, we start from scratch
if len(loaded_ids) == 0:
    # Drop the table if it exists
    cursor.execute("DROP TABLE IF EXISTS state_vector_weather;")

    # Create the new table
    create_table_query = f'''
        CREATE TABLE state_vector_weather (
            vector_id INTEGER PRIMARY KEY,
            {", ".join([f"{col} REAL" for col in new_columns])}
        );
    '''
    # Create the new table if it doesn't exist
    cursor.execute(create_table_query)

# Commits change to file
flights_connection.commit()

t_start_full = datetime.datetime.now()
# Looping through flight ids
for i, flight_id in enumerate(flight_ids):
    time_iteration = datetime.datetime.now()
    # Clears output at every loop
    clear_output(wait=True)
    print(f'{flight_id} | {i}/{len(flight_ids)}.')
    time_elapsed = (time_iteration - t_start_full).total_seconds()
    if i == 0:
        ETA = np.nan
    else:
        ETA = time_elapsed*len(flight_ids)/i - time_elapsed
    print(f'Time Elapsed: {utils.format_time(time_elapsed)}.')
    print(f'Estimate time to finish: {utils.format_time(ETA)}.')
    
    # Finds minimum and maximum flight for current flight
    print('Loading time limits.')
    min_time, max_time = pd.read_sql_query(f"""
                                SELECT MIN(time) as min_time, MAX(time) as max_time
                                FROM state_vectors 
                                JOIN flights ON flights.flight_id = state_vectors.flight_id
                                WHERE state_vectors.flight_id = "{flight_id}";
                               """,
                              flights_connection
                              ).values[0]
    
    # Finds the minimum and maximum latitudes for the current flight
    print('Loading latitude limits.')
    min_latitude, max_latitude = pd.read_sql_query(f"""
                                    SELECT MIN(lat) as min_lat, MAX(lat) as max_lat
                                    FROM state_vectors 
                                    JOIN flights ON flights.flight_id = state_vectors.flight_id
                                    WHERE state_vectors.flight_id = "{flight_id}"
                                   """,
                                  flights_connection
                                  ).values[0]

    print('Loading longitude limits.')
    # Finds the minimum and maximum longitudes for the current flight
    min_longitude, max_longitude = pd.read_sql_query(f"""
                                    SELECT MIN(lon) as min_lon, MAX(lon) as max_lon
                                    FROM state_vectors 
                                    JOIN flights ON flights.flight_id = state_vectors.flight_id
                                    WHERE state_vectors.flight_id = "{flight_id}"
                                   """,
                                  flights_connection
                                  ).values[0]
    
    # Adjusting time, lat and lon thresholds.
    print("Adjusting time, lat, lon thresholds.")
    # Adds time threshold to time limits
    min_time -= time_thresh
    max_time += time_thresh
    
    # Adds latitude and longitude threshold
    range_latitude = max_latitude - min_latitude
    range_longitude = max_longitude - min_longitude
    min_latitude -= range_latitude*lat_lon_thresh
    max_latitude += range_latitude*lat_lon_thresh
    min_longitude -= range_longitude*lat_lon_thresh
    max_longitude += range_longitude*lat_lon_thresh    
    
    # Loads the weather data corresponding to the flight
    print('Loading relevant weather data.')
    flight_weather_data = pd.read_sql_query(f"""
                                    SELECT ws.lat, ws.lon, ws.elevation, ws.sigma, wd.*
                                    FROM weather_data as wd
                                    JOIN weather_stations as ws ON ws.station_id = wd.station_id
                                    WHERE wd.time BETWEEN {min_time} AND {max_time};
                                   """,
                                   weather_connection
                                    )

    # Loads state vectors for the given flight
    print('Loading state vectors.')
    state_vectors = pd.read_sql_query(f"""
                                    SELECT DISTINCT state_vectors.*
                                    FROM state_vectors 
                                    JOIN flights ON flights.flight_id = state_vectors.flight_id
                                    WHERE state_vectors.flight_id = "{flight_id}";
                                   """,
                                   flights_connection)
    
    # Computes the weather values for the current flight
    print('Computing weather values.')
    state_vectors = interpolator.compute_flight_weather_quantities(new_columns, state_vectors, stations_data=flight_weather_data)
    
    # Adds newly calculated values to new table    
    print("Adding newly calculated values to new table")
    for index, row in state_vectors.iterrows():
        # Preparing the data to be inserted
        insert_data = tuple(row[col] for col in ['vector_id'] + new_columns)

        # Creating query to insert new values
        insert_query = f'''
            INSERT INTO state_vector_weather (vector_id, {', '.join(new_columns)})
            VALUES ({', '.join('?' * len(insert_data))})
            ON CONFLICT(vector_id) DO UPDATE SET
            {', '.join([f"{col} = excluded.{col}" for col in new_columns])};
        '''
        cursor.execute(insert_query, insert_data)
    # Commiting changes to the database
    flights_connection.commit()
    
    with open(tracking_file, 'a') as f:
        f.write(flight_id + '\n')

# Closing connections
flights_connection.close()
weather_connection.close()

print("Done!")

a12c18_1690481880_1690500176_KJFK_KLAX | 998/999.
Time Elapsed: 1d 3h 53m 49s.
Estimate time to finish: 1m 40s.
Loading time limits.
Loading latitude limits.
Loading longitude limits.
Adjusting time, lat, lon thresholds.
Loading relevant weather data.
Loading state vectors.
Computing weather values.
Adding newly calculated values to new table
Done!


In [19]:
import sys
# This variable should indicate the path from this Jupyter Notebook to the root directory of the repo.
root_path = '../'
# Adds the repo's root to the list of paths
sys.path.append(root_path)

# Package to read yml files
import yaml
# Package to handle file paths
import os
# Package to deal with DataFrames
import pandas as pd
# Package to plot stuff
import matplotlib.pyplot as plt
# Package for numerical and array handling
import numpy as np
# Package to read and write to .sqlite files
import sqlite3
# Package to keep track of time
import datetime

# Function to clear output from jupyter notebook
from IPython.display import clear_output
# Package for compressing dataframes into file
from src.data import compressors
# Package for defining and fitting weather models
from src.models import weather
# Utilities package
from src.common import utils
# Package for interpolating and estimating weather
from src.analysis import weather_interpolator

# Path from this notebook to the root directory
root_path = os.path.normpath(root_path)
# Path from root to the desired config file
config_path_from_root = os.path.normpath('config/config.yml')
# Defining path from this notebook to config file
config_path = os.path.join(root_path, config_path_from_root)

# Loading config file
with open(config_path, 'r',  encoding='utf8') as file:
    config = yaml.safe_load(file)

# Defining "clear-output" function to feed into logger
def clear():
    clear_output(wait=True)

# Creates an instance of a logger class to log all that happens, optional (but encouraged).
logger = utils.Logger(config, clear_function=None)

# Creates an instance of the weather interpolator
interpolator = weather_interpolator.WeatherInterpolator(config, logger=logger)

# Defining location of data
flights_database = '../data/flight/KLAX_KJFK_2023-07-01_2023-07-31.sqlite'
weather_database = '../data/weather/Weather-US_2023-06-30_2022-08-01.sqlite'

# Path to file keeping track of already-loaded flights
tracking_file = flights_database.replace('sqlite','txt')

In [20]:
# Declares the connections read from each of the databases
flights_connection = sqlite3.connect(flights_database)
weather_connection = sqlite3.connect(weather_database)
# Declares a cursor to write to the database
cursor = flights_connection.cursor()

# Checking and loading the tracking_file
if os.path.isfile(tracking_file):
    with open(tracking_file, 'r') as f:
        loaded_ids = f.read().split('\n')
        loaded_ids = [i for i in loaded_ids if i != '']
else:
    with open(tracking_file, 'w') as f:
        loaded_ids = []
    

# Runs a query to identify all the flight_ids available
flight_ids = pd.read_sql_query("SELECT flight_id FROM flights;", flights_connection).values[:,0]
# Only care about the ids that have not been loaded yet.
flight_ids = [f for f in flight_ids if f not in loaded_ids]

# Loading the time threshold variable, which is the time interval that weather data will be loaded for each calculation.
time_thresh = config['statistics']['interpolation']['weather']['time-thresh']
lat_lon_thresh = config['statistics']['interpolation']['weather']['lat-lon-thresh']

# The list of columns to be added to the new table to be created.
new_columns = ['tmpf', 'air_pressure', 'air_density', 'clouds', 'sknt', 'severity']

# If there is no record of loaded ids, we start from scratch
if len(loaded_ids) == 0:
    # Drop the table if it exists
    cursor.execute("DROP TABLE IF EXISTS state_vector_weather;")

    # Create the new table
    create_table_query = f'''
        CREATE TABLE state_vector_weather (
            vector_id INTEGER PRIMARY KEY,
            {", ".join([f"{col} REAL" for col in new_columns])}
        );
    '''
    # Create the new table if it doesn't exist
    cursor.execute(create_table_query)

# Commits change to file
flights_connection.commit()

t_start_full = datetime.datetime.now()
# Looping through flight ids
for i, flight_id in enumerate(flight_ids):
    time_iteration = datetime.datetime.now()
    # Clears output at every loop
    clear_output(wait=True)
    print(f'{flight_id} | {i}/{len(flight_ids)}.')
    time_elapsed = (time_iteration - t_start_full).total_seconds()
    if i == 0:
        ETA = np.nan
    else:
        ETA = time_elapsed*len(flight_ids)/i - time_elapsed
    print(f'Time Elapsed: {utils.format_time(time_elapsed)}.')
    print(f'Estimate time to finish: {utils.format_time(ETA)}.')
    
    # Finds minimum and maximum flight for current flight
    print('Loading time limits.')
    min_time, max_time = pd.read_sql_query(f"""
                                SELECT MIN(time) as min_time, MAX(time) as max_time
                                FROM state_vectors 
                                JOIN flights ON flights.flight_id = state_vectors.flight_id
                                WHERE state_vectors.flight_id = "{flight_id}";
                               """,
                              flights_connection
                              ).values[0]
    
    # Finds the minimum and maximum latitudes for the current flight
    print('Loading latitude limits.')
    min_latitude, max_latitude = pd.read_sql_query(f"""
                                    SELECT MIN(lat) as min_lat, MAX(lat) as max_lat
                                    FROM state_vectors 
                                    JOIN flights ON flights.flight_id = state_vectors.flight_id
                                    WHERE state_vectors.flight_id = "{flight_id}"
                                   """,
                                  flights_connection
                                  ).values[0]

    print('Loading longitude limits.')
    # Finds the minimum and maximum longitudes for the current flight
    min_longitude, max_longitude = pd.read_sql_query(f"""
                                    SELECT MIN(lon) as min_lon, MAX(lon) as max_lon
                                    FROM state_vectors 
                                    JOIN flights ON flights.flight_id = state_vectors.flight_id
                                    WHERE state_vectors.flight_id = "{flight_id}"
                                   """,
                                  flights_connection
                                  ).values[0]
    
    # Adjusting time, lat and lon thresholds.
    print("Adjusting time, lat, lon thresholds.")
    # Adds time threshold to time limits
    min_time -= time_thresh
    max_time += time_thresh
    
    # Adds latitude and longitude threshold
    range_latitude = max_latitude - min_latitude
    range_longitude = max_longitude - min_longitude
    min_latitude -= range_latitude*lat_lon_thresh
    max_latitude += range_latitude*lat_lon_thresh
    min_longitude -= range_longitude*lat_lon_thresh
    max_longitude += range_longitude*lat_lon_thresh    
    
    # Loads the weather data corresponding to the flight
    print('Loading relevant weather data.')
    flight_weather_data = pd.read_sql_query(f"""
                                    SELECT ws.lat, ws.lon, ws.elevation, ws.sigma, wd.*
                                    FROM weather_data as wd
                                    JOIN weather_stations as ws ON ws.station_id = wd.station_id
                                    WHERE wd.time BETWEEN {min_time} AND {max_time};
                                   """,
                                   weather_connection
                                    )

    # Loads state vectors for the given flight
    print('Loading state vectors.')
    state_vectors = pd.read_sql_query(f"""
                                    SELECT DISTINCT state_vectors.*
                                    FROM state_vectors 
                                    JOIN flights ON flights.flight_id = state_vectors.flight_id
                                    WHERE state_vectors.flight_id = "{flight_id}";
                                   """,
                                   flights_connection)
    
    # Computes the weather values for the current flight
    print('Computing weather values.')
    state_vectors = interpolator.compute_flight_weather_quantities(new_columns, state_vectors, stations_data=flight_weather_data)
    
    # Adds newly calculated values to new table    
    print("Adding newly calculated values to new table")
    for index, row in state_vectors.iterrows():
        # Preparing the data to be inserted
        insert_data = tuple(row[col] for col in ['vector_id'] + new_columns)

        # Creating query to insert new values
        insert_query = f'''
            INSERT INTO state_vector_weather (vector_id, {', '.join(new_columns)})
            VALUES ({', '.join('?' * len(insert_data))})
            ON CONFLICT(vector_id) DO UPDATE SET
            {', '.join([f"{col} = excluded.{col}" for col in new_columns])};
        '''
        cursor.execute(insert_query, insert_data)
    # Commiting changes to the database
    flights_connection.commit()
    
    with open(tracking_file, 'a') as f:
        f.write(flight_id + '\n')

# Closing connections
flights_connection.close()
weather_connection.close()

print("Done!")

a11380_1690482460_1690501195_KLAX_KJFK | 966/967.
Time Elapsed: 1d 2h 7m 1s.
Estimate time to finish: 1m 37s.
Loading time limits.
Loading latitude limits.
Loading longitude limits.
Adjusting time, lat, lon thresholds.
Loading relevant weather data.
Loading state vectors.
Computing weather values.
Adding newly calculated values to new table
Done!


In [21]:
import sys
# This variable should indicate the path from this Jupyter Notebook to the root directory of the repo.
root_path = '../'
# Adds the repo's root to the list of paths
sys.path.append(root_path)

# Package to read yml files
import yaml
# Package to handle file paths
import os
# Package to deal with DataFrames
import pandas as pd
# Package to plot stuff
import matplotlib.pyplot as plt
# Package for numerical and array handling
import numpy as np
# Package to read and write to .sqlite files
import sqlite3
# Package to keep track of time
import datetime

# Function to clear output from jupyter notebook
from IPython.display import clear_output
# Package for compressing dataframes into file
from src.data import compressors
# Package for defining and fitting weather models
from src.models import weather
# Utilities package
from src.common import utils
# Package for interpolating and estimating weather
from src.analysis import weather_interpolator

# Path from this notebook to the root directory
root_path = os.path.normpath(root_path)
# Path from root to the desired config file
config_path_from_root = os.path.normpath('config/config.yml')
# Defining path from this notebook to config file
config_path = os.path.join(root_path, config_path_from_root)

# Loading config file
with open(config_path, 'r',  encoding='utf8') as file:
    config = yaml.safe_load(file)

# Defining "clear-output" function to feed into logger
def clear():
    clear_output(wait=True)

# Creates an instance of a logger class to log all that happens, optional (but encouraged).
logger = utils.Logger(config, clear_function=None)

# Creates an instance of the weather interpolator
interpolator = weather_interpolator.WeatherInterpolator(config, logger=logger)

# Defining location of data
flights_database = '../data/flight/KLAX_KSFO_2023-07-01_2023-07-31.sqlite'
weather_database = '../data/weather/Weather-US_2023-06-30_2022-08-01.sqlite'

# Path to file keeping track of already-loaded flights
tracking_file = flights_database.replace('sqlite','txt')

In [22]:
# Declares the connections read from each of the databases
flights_connection = sqlite3.connect(flights_database)
weather_connection = sqlite3.connect(weather_database)
# Declares a cursor to write to the database
cursor = flights_connection.cursor()

# Checking and loading the tracking_file
if os.path.isfile(tracking_file):
    with open(tracking_file, 'r') as f:
        loaded_ids = f.read().split('\n')
        loaded_ids = [i for i in loaded_ids if i != '']
else:
    with open(tracking_file, 'w') as f:
        loaded_ids = []
    

# Runs a query to identify all the flight_ids available
flight_ids = pd.read_sql_query("SELECT flight_id FROM flights;", flights_connection).values[:,0]
# Only care about the ids that have not been loaded yet.
flight_ids = [f for f in flight_ids if f not in loaded_ids]

# Loading the time threshold variable, which is the time interval that weather data will be loaded for each calculation.
time_thresh = config['statistics']['interpolation']['weather']['time-thresh']
lat_lon_thresh = config['statistics']['interpolation']['weather']['lat-lon-thresh']

# The list of columns to be added to the new table to be created.
new_columns = ['tmpf', 'air_pressure', 'air_density', 'clouds', 'sknt', 'severity']

# If there is no record of loaded ids, we start from scratch
if len(loaded_ids) == 0:
    # Drop the table if it exists
    cursor.execute("DROP TABLE IF EXISTS state_vector_weather;")

    # Create the new table
    create_table_query = f'''
        CREATE TABLE state_vector_weather (
            vector_id INTEGER PRIMARY KEY,
            {", ".join([f"{col} REAL" for col in new_columns])}
        );
    '''
    # Create the new table if it doesn't exist
    cursor.execute(create_table_query)

# Commits change to file
flights_connection.commit()

t_start_full = datetime.datetime.now()
# Looping through flight ids
for i, flight_id in enumerate(flight_ids):
    time_iteration = datetime.datetime.now()
    # Clears output at every loop
    clear_output(wait=True)
    print(f'{flight_id} | {i}/{len(flight_ids)}.')
    time_elapsed = (time_iteration - t_start_full).total_seconds()
    if i == 0:
        ETA = np.nan
    else:
        ETA = time_elapsed*len(flight_ids)/i - time_elapsed
    print(f'Time Elapsed: {utils.format_time(time_elapsed)}.')
    print(f'Estimate time to finish: {utils.format_time(ETA)}.')
    
    # Finds minimum and maximum flight for current flight
    print('Loading time limits.')
    min_time, max_time = pd.read_sql_query(f"""
                                SELECT MIN(time) as min_time, MAX(time) as max_time
                                FROM state_vectors 
                                JOIN flights ON flights.flight_id = state_vectors.flight_id
                                WHERE state_vectors.flight_id = "{flight_id}";
                               """,
                              flights_connection
                              ).values[0]
    
    # Finds the minimum and maximum latitudes for the current flight
    print('Loading latitude limits.')
    min_latitude, max_latitude = pd.read_sql_query(f"""
                                    SELECT MIN(lat) as min_lat, MAX(lat) as max_lat
                                    FROM state_vectors 
                                    JOIN flights ON flights.flight_id = state_vectors.flight_id
                                    WHERE state_vectors.flight_id = "{flight_id}"
                                   """,
                                  flights_connection
                                  ).values[0]

    print('Loading longitude limits.')
    # Finds the minimum and maximum longitudes for the current flight
    min_longitude, max_longitude = pd.read_sql_query(f"""
                                    SELECT MIN(lon) as min_lon, MAX(lon) as max_lon
                                    FROM state_vectors 
                                    JOIN flights ON flights.flight_id = state_vectors.flight_id
                                    WHERE state_vectors.flight_id = "{flight_id}"
                                   """,
                                  flights_connection
                                  ).values[0]
    
    # Adjusting time, lat and lon thresholds.
    print("Adjusting time, lat, lon thresholds.")
    # Adds time threshold to time limits
    min_time -= time_thresh
    max_time += time_thresh
    
    # Adds latitude and longitude threshold
    range_latitude = max_latitude - min_latitude
    range_longitude = max_longitude - min_longitude
    min_latitude -= range_latitude*lat_lon_thresh
    max_latitude += range_latitude*lat_lon_thresh
    min_longitude -= range_longitude*lat_lon_thresh
    max_longitude += range_longitude*lat_lon_thresh    
    
    # Loads the weather data corresponding to the flight
    print('Loading relevant weather data.')
    flight_weather_data = pd.read_sql_query(f"""
                                    SELECT ws.lat, ws.lon, ws.elevation, ws.sigma, wd.*
                                    FROM weather_data as wd
                                    JOIN weather_stations as ws ON ws.station_id = wd.station_id
                                    WHERE wd.time BETWEEN {min_time} AND {max_time};
                                   """,
                                   weather_connection
                                    )

    # Loads state vectors for the given flight
    print('Loading state vectors.')
    state_vectors = pd.read_sql_query(f"""
                                    SELECT DISTINCT state_vectors.*
                                    FROM state_vectors 
                                    JOIN flights ON flights.flight_id = state_vectors.flight_id
                                    WHERE state_vectors.flight_id = "{flight_id}";
                                   """,
                                   flights_connection)
    
    # Computes the weather values for the current flight
    print('Computing weather values.')
    state_vectors = interpolator.compute_flight_weather_quantities(new_columns, state_vectors, stations_data=flight_weather_data)
    
    # Adds newly calculated values to new table    
    print("Adding newly calculated values to new table")
    for index, row in state_vectors.iterrows():
        # Preparing the data to be inserted
        insert_data = tuple(row[col] for col in ['vector_id'] + new_columns)

        # Creating query to insert new values
        insert_query = f'''
            INSERT INTO state_vector_weather (vector_id, {', '.join(new_columns)})
            VALUES ({', '.join('?' * len(insert_data))})
            ON CONFLICT(vector_id) DO UPDATE SET
            {', '.join([f"{col} = excluded.{col}" for col in new_columns])};
        '''
        cursor.execute(insert_query, insert_data)
    # Commiting changes to the database
    flights_connection.commit()
    
    with open(tracking_file, 'a') as f:
        f.write(flight_id + '\n')

# Closing connections
flights_connection.close()
weather_connection.close()

print("Done!")

OperationalError: database is locked

In [ ]:
import sys
# This variable should indicate the path from this Jupyter Notebook to the root directory of the repo.
root_path = '../'
# Adds the repo's root to the list of paths
sys.path.append(root_path)

# Package to read yml files
import yaml
# Package to handle file paths
import os
# Package to deal with DataFrames
import pandas as pd
# Package to plot stuff
import matplotlib.pyplot as plt
# Package for numerical and array handling
import numpy as np
# Package to read and write to .sqlite files
import sqlite3
# Package to keep track of time
import datetime

# Function to clear output from jupyter notebook
from IPython.display import clear_output
# Package for compressing dataframes into file
from src.data import compressors
# Package for defining and fitting weather models
from src.models import weather
# Utilities package
from src.common import utils
# Package for interpolating and estimating weather
from src.analysis import weather_interpolator

# Path from this notebook to the root directory
root_path = os.path.normpath(root_path)
# Path from root to the desired config file
config_path_from_root = os.path.normpath('config/config.yml')
# Defining path from this notebook to config file
config_path = os.path.join(root_path, config_path_from_root)

# Loading config file
with open(config_path, 'r',  encoding='utf8') as file:
    config = yaml.safe_load(file)

# Defining "clear-output" function to feed into logger
def clear():
    clear_output(wait=True)

# Creates an instance of a logger class to log all that happens, optional (but encouraged).
logger = utils.Logger(config, clear_function=None)

# Creates an instance of the weather interpolator
interpolator = weather_interpolator.WeatherInterpolator(config, logger=logger)

# Defining location of data
flights_database = '../data/flight/KSEA_KDEN_2023-07-01_2023-07-31.sqlite'
weather_database = '../data/weather/Weather-US_2023-06-30_2022-08-01.sqlite'

# Path to file keeping track of already-loaded flights
tracking_file = flights_database.replace('sqlite','txt')

In [ ]:
# Declares the connections read from each of the databases
flights_connection = sqlite3.connect(flights_database)
weather_connection = sqlite3.connect(weather_database)
# Declares a cursor to write to the database
cursor = flights_connection.cursor()

# Checking and loading the tracking_file
if os.path.isfile(tracking_file):
    with open(tracking_file, 'r') as f:
        loaded_ids = f.read().split('\n')
        loaded_ids = [i for i in loaded_ids if i != '']
else:
    with open(tracking_file, 'w') as f:
        loaded_ids = []
    

# Runs a query to identify all the flight_ids available
flight_ids = pd.read_sql_query("SELECT flight_id FROM flights;", flights_connection).values[:,0]
# Only care about the ids that have not been loaded yet.
flight_ids = [f for f in flight_ids if f not in loaded_ids]

# Loading the time threshold variable, which is the time interval that weather data will be loaded for each calculation.
time_thresh = config['statistics']['interpolation']['weather']['time-thresh']
lat_lon_thresh = config['statistics']['interpolation']['weather']['lat-lon-thresh']

# The list of columns to be added to the new table to be created.
new_columns = ['tmpf', 'air_pressure', 'air_density', 'clouds', 'sknt', 'severity']

# If there is no record of loaded ids, we start from scratch
if len(loaded_ids) == 0:
    # Drop the table if it exists
    cursor.execute("DROP TABLE IF EXISTS state_vector_weather;")

    # Create the new table
    create_table_query = f'''
        CREATE TABLE state_vector_weather (
            vector_id INTEGER PRIMARY KEY,
            {", ".join([f"{col} REAL" for col in new_columns])}
        );
    '''
    # Create the new table if it doesn't exist
    cursor.execute(create_table_query)

# Commits change to file
flights_connection.commit()

t_start_full = datetime.datetime.now()
# Looping through flight ids
for i, flight_id in enumerate(flight_ids):
    time_iteration = datetime.datetime.now()
    # Clears output at every loop
    clear_output(wait=True)
    print(f'{flight_id} | {i}/{len(flight_ids)}.')
    time_elapsed = (time_iteration - t_start_full).total_seconds()
    if i == 0:
        ETA = np.nan
    else:
        ETA = time_elapsed*len(flight_ids)/i - time_elapsed
    print(f'Time Elapsed: {utils.format_time(time_elapsed)}.')
    print(f'Estimate time to finish: {utils.format_time(ETA)}.')
    
    # Finds minimum and maximum flight for current flight
    print('Loading time limits.')
    min_time, max_time = pd.read_sql_query(f"""
                                SELECT MIN(time) as min_time, MAX(time) as max_time
                                FROM state_vectors 
                                JOIN flights ON flights.flight_id = state_vectors.flight_id
                                WHERE state_vectors.flight_id = "{flight_id}";
                               """,
                              flights_connection
                              ).values[0]
    
    # Finds the minimum and maximum latitudes for the current flight
    print('Loading latitude limits.')
    min_latitude, max_latitude = pd.read_sql_query(f"""
                                    SELECT MIN(lat) as min_lat, MAX(lat) as max_lat
                                    FROM state_vectors 
                                    JOIN flights ON flights.flight_id = state_vectors.flight_id
                                    WHERE state_vectors.flight_id = "{flight_id}"
                                   """,
                                  flights_connection
                                  ).values[0]

    print('Loading longitude limits.')
    # Finds the minimum and maximum longitudes for the current flight
    min_longitude, max_longitude = pd.read_sql_query(f"""
                                    SELECT MIN(lon) as min_lon, MAX(lon) as max_lon
                                    FROM state_vectors 
                                    JOIN flights ON flights.flight_id = state_vectors.flight_id
                                    WHERE state_vectors.flight_id = "{flight_id}"
                                   """,
                                  flights_connection
                                  ).values[0]
    
    # Adjusting time, lat and lon thresholds.
    print("Adjusting time, lat, lon thresholds.")
    # Adds time threshold to time limits
    min_time -= time_thresh
    max_time += time_thresh
    
    # Adds latitude and longitude threshold
    range_latitude = max_latitude - min_latitude
    range_longitude = max_longitude - min_longitude
    min_latitude -= range_latitude*lat_lon_thresh
    max_latitude += range_latitude*lat_lon_thresh
    min_longitude -= range_longitude*lat_lon_thresh
    max_longitude += range_longitude*lat_lon_thresh    
    
    # Loads the weather data corresponding to the flight
    print('Loading relevant weather data.')
    flight_weather_data = pd.read_sql_query(f"""
                                    SELECT ws.lat, ws.lon, ws.elevation, ws.sigma, wd.*
                                    FROM weather_data as wd
                                    JOIN weather_stations as ws ON ws.station_id = wd.station_id
                                    WHERE wd.time BETWEEN {min_time} AND {max_time};
                                   """,
                                   weather_connection
                                    )

    # Loads state vectors for the given flight
    print('Loading state vectors.')
    state_vectors = pd.read_sql_query(f"""
                                    SELECT DISTINCT state_vectors.*
                                    FROM state_vectors 
                                    JOIN flights ON flights.flight_id = state_vectors.flight_id
                                    WHERE state_vectors.flight_id = "{flight_id}";
                                   """,
                                   flights_connection)
    
    # Computes the weather values for the current flight
    print('Computing weather values.')
    state_vectors = interpolator.compute_flight_weather_quantities(new_columns, state_vectors, stations_data=flight_weather_data)
    
    # Adds newly calculated values to new table    
    print("Adding newly calculated values to new table")
    for index, row in state_vectors.iterrows():
        # Preparing the data to be inserted
        insert_data = tuple(row[col] for col in ['vector_id'] + new_columns)

        # Creating query to insert new values
        insert_query = f'''
            INSERT INTO state_vector_weather (vector_id, {', '.join(new_columns)})
            VALUES ({', '.join('?' * len(insert_data))})
            ON CONFLICT(vector_id) DO UPDATE SET
            {', '.join([f"{col} = excluded.{col}" for col in new_columns])};
        '''
        cursor.execute(insert_query, insert_data)
    # Commiting changes to the database
    flights_connection.commit()
    
    with open(tracking_file, 'a') as f:
        f.write(flight_id + '\n')

# Closing connections
flights_connection.close()
weather_connection.closeose()

print("Done!")

In [ ]:
import sys
# This variable should indicate the path from this Jupyter Notebook to the root directory of the repo.
root_path = '../'
# Adds the repo's root to the list of paths
sys.path.append(root_path)

# Package to read yml files
import yaml
# Package to handle file paths
import os
# Package to deal with DataFrames
import pandas as pd
# Package to plot stuff
import matplotlib.pyplot as plt
# Package for numerical and array handling
import numpy as np
# Package to read and write to .sqlite files
import sqlite3
# Package to keep track of time
import datetime

# Function to clear output from jupyter notebook
from IPython.display import clear_output
# Package for compressing dataframes into file
from src.data import compressors
# Package for defining and fitting weather models
from src.models import weather
# Utilities package
from src.common import utils
# Package for interpolating and estimating weather
from src.analysis import weather_interpolator

# Path from this notebook to the root directory
root_path = os.path.normpath(root_path)
# Path from root to the desired config file
config_path_from_root = os.path.normpath('config/config.yml')
# Defining path from this notebook to config file
config_path = os.path.join(root_path, config_path_from_root)

# Loading config file
with open(config_path, 'r',  encoding='utf8') as file:
    config = yaml.safe_load(file)

# Defining "clear-output" function to feed into logger
def clear():
    clear_output(wait=True)

# Creates an instance of a logger class to log all that happens, optional (but encouraged).
logger = utils.Logger(config, clear_function=None)

# Creates an instance of the weather interpolator
interpolator = weather_interpolator.WeatherInterpolator(config, logger=logger)

# Defining location of data
flights_database = '../data/flight/KSFO_KLAX_2023-07-01_2023-07-31.sqlite'
weather_database = '../data/weather/Weather-US_2023-06-30_2022-08-01.sqlite'

# Path to file keeping track of already-loaded flights
tracking_file = flights_database.replace('sqlite','txt')

In [ ]:
# Declares the connections read from each of the databases
flights_connection = sqlite3.connect(flights_database)
weather_connection = sqlite3.connect(weather_database)
# Declares a cursor to write to the database
cursor = flights_connection.cursor()

# Checking and loading the tracking_file
if os.path.isfile(tracking_file):
    with open(tracking_file, 'r') as f:
        loaded_ids = f.read().split('\n')
        loaded_ids = [i for i in loaded_ids if i != '']
else:
    with open(tracking_file, 'w') as f:
        loaded_ids = []
    

# Runs a query to identify all the flight_ids available
flight_ids = pd.read_sql_query("SELECT flight_id FROM flights;", flights_connection).values[:,0]
# Only care about the ids that have not been loaded yet.
flight_ids = [f for f in flight_ids if f not in loaded_ids]

# Loading the time threshold variable, which is the time interval that weather data will be loaded for each calculation.
time_thresh = config['statistics']['interpolation']['weather']['time-thresh']
lat_lon_thresh = config['statistics']['interpolation']['weather']['lat-lon-thresh']

# The list of columns to be added to the new table to be created.
new_columns = ['tmpf', 'air_pressure', 'air_density', 'clouds', 'sknt', 'severity']

# If there is no record of loaded ids, we start from scratch
if len(loaded_ids) == 0:
    # Drop the table if it exists
    cursor.execute("DROP TABLE IF EXISTS state_vector_weather;")

    # Create the new table
    create_table_query = f'''
        CREATE TABLE state_vector_weather (
            vector_id INTEGER PRIMARY KEY,
            {", ".join([f"{col} REAL" for col in new_columns])}
        );
    '''
    # Create the new table if it doesn't exist
    cursor.execute(create_table_query)

# Commits change to file
flights_connection.commit()

t_start_full = datetime.datetime.now()
# Looping through flight ids
for i, flight_id in enumerate(flight_ids):
    time_iteration = datetime.datetime.now()
    # Clears output at every loop
    clear_output(wait=True)
    print(f'{flight_id} | {i}/{len(flight_ids)}.')
    time_elapsed = (time_iteration - t_start_full).total_seconds()
    if i == 0:
        ETA = np.nan
    else:
        ETA = time_elapsed*len(flight_ids)/i - time_elapsed
    print(f'Time Elapsed: {utils.format_time(time_elapsed)}.')
    print(f'Estimate time to finish: {utils.format_time(ETA)}.')
    
    # Finds minimum and maximum flight for current flight
    print('Loading time limits.')
    min_time, max_time = pd.read_sql_query(f"""
                                SELECT MIN(time) as min_time, MAX(time) as max_time
                                FROM state_vectors 
                                JOIN flights ON flights.flight_id = state_vectors.flight_id
                                WHERE state_vectors.flight_id = "{flight_id}";
                               """,
                              flights_connection
                              ).values[0]
    
    # Finds the minimum and maximum latitudes for the current flight
    print('Loading latitude limits.')
    min_latitude, max_latitude = pd.read_sql_query(f"""
                                    SELECT MIN(lat) as min_lat, MAX(lat) as max_lat
                                    FROM state_vectors 
                                    JOIN flights ON flights.flight_id = state_vectors.flight_id
                                    WHERE state_vectors.flight_id = "{flight_id}"
                                   """,
                                  flights_connection
                                  ).values[0]

    print('Loading longitude limits.')
    # Finds the minimum and maximum longitudes for the current flight
    min_longitude, max_longitude = pd.read_sql_query(f"""
                                    SELECT MIN(lon) as min_lon, MAX(lon) as max_lon
                                    FROM state_vectors 
                                    JOIN flights ON flights.flight_id = state_vectors.flight_id
                                    WHERE state_vectors.flight_id = "{flight_id}"
                                   """,
                                  flights_connection
                                  ).values[0]
    
    # Adjusting time, lat and lon thresholds.
    print("Adjusting time, lat, lon thresholds.")
    # Adds time threshold to time limits
    min_time -= time_thresh
    max_time += time_thresh
    
    # Adds latitude and longitude threshold
    range_latitude = max_latitude - min_latitude
    range_longitude = max_longitude - min_longitude
    min_latitude -= range_latitude*lat_lon_thresh
    max_latitude += range_latitude*lat_lon_thresh
    min_longitude -= range_longitude*lat_lon_thresh
    max_longitude += range_longitude*lat_lon_thresh    
    
    # Loads the weather data corresponding to the flight
    print('Loading relevant weather data.')
    flight_weather_data = pd.read_sql_query(f"""
                                    SELECT ws.lat, ws.lon, ws.elevation, ws.sigma, wd.*
                                    FROM weather_data as wd
                                    JOIN weather_stations as ws ON ws.station_id = wd.station_id
                                    WHERE wd.time BETWEEN {min_time} AND {max_time};
                                   """,
                                   weather_connection
                                    )

    # Loads state vectors for the given flight
    print('Loading state vectors.')
    state_vectors = pd.read_sql_query(f"""
                                    SELECT DISTINCT state_vectors.*
                                    FROM state_vectors 
                                    JOIN flights ON flights.flight_id = state_vectors.flight_id
                                    WHERE state_vectors.flight_id = "{flight_id}";
                                   """,
                                   flights_connection)
    
    # Computes the weather values for the current flight
    print('Computing weather values.')
    state_vectors = interpolator.compute_flight_weather_quantities(new_columns, state_vectors, stations_data=flight_weather_data)
    
    # Adds newly calculated values to new table    
    print("Adding newly calculated values to new table")
    for index, row in state_vectors.iterrows():
        # Preparing the data to be inserted
        insert_data = tuple(row[col] for col in ['vector_id'] + new_columns)

        # Creating query to insert new values
        insert_query = f'''
            INSERT INTO state_vector_weather (vector_id, {', '.join(new_columns)})
            VALUES ({', '.join('?' * len(insert_data))})
            ON CONFLICT(vector_id) DO UPDATE SET
            {', '.join([f"{col} = excluded.{col}" for col in new_columns])};
        '''
        cursor.execute(insert_query, insert_data)
    # Commiting changes to the database
    flights_connection.commit()
    
    with open(tracking_file, 'a') as f:
        f.write(flight_id + '\n')

# Closing connections
flights_connection.close()
weather_connection.close()

print("Done!")